In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import pandas as pd
#csv_file = '/content/drive/MyDrive/ML_FDP_hands_on_2024/ADReSS_train.csv'
#csv_file = pd.read_csv(csv_file)
#df = csv_file.sample(frac = 1)
#filepath = '/content/drive/MyDrive/ML_FDP_hands_on_2024/ADReSS_train_shuffle_3.csv'
#df.to_csv(filepath, index=False)


In [9]:
import torch
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pdb import set_trace as bp




class SimpleNet(nn.Module):
    def __init__(self, input_size):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(128, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        #x = self.relu(self.fc2(x))
        #x = self.relu(self.fc3(x))
        return self.fc4(x)

class AudioDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        label = torch.tensor(int(self.data_frame.iloc[idx, 0]), dtype=torch.long)
        #bp()
        features = torch.tensor(self.data_frame.iloc[idx, 1:].values, dtype=torch.float32)

        if self.transform:
            label, features = self.transform((label, features))

        sample = {'label': label, 'features': features}
        #bp()
        return sample

#def calculate_accuracy(outputs, labels):
#    _, predicted = torch.max(outputs.data, 1)
#    total = labels.size(0)
#    correct = (predicted == labels).sum().item()
#    accuracy = correct / total
#    return accuracy

# Separate train and test sets
#train_set = AudioDataset(csv_file='/content/drive/MyDrive/ML_FDP_hands_on_2024/ADReSS_train_shuffle_3.csv')
train_set = AudioDataset(csv_file='/content/drive/MyDrive/ML_FDP_hands_on_2024/ADReSS_train.csv')

test_set = AudioDataset(csv_file='/content/drive/MyDrive/ML_FDP_hands_on_2024/ADReSS_test.csv')

# DataLoader for training and testing
train_loader = DataLoader(train_set, batch_size=4, shuffle=False)
test_loader = DataLoader(test_set, batch_size=4, shuffle=False)

# Instantiate the neural network
input_size = 19  # Number of MFCC features
model = SimpleNet(input_size)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_train = 0
    total_train = 0

    for batch in train_loader:
        label, features = batch['label'], batch['features']
        #bp()
        lrn_rate = get_lr(optimizer)
        # Forward pass
        outputs = model(features)

        # Compute loss
        loss = criterion(outputs, label)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lrn_rate = get_lr(optimizer)

        # Update training accuracy
        total_train += label.size(0)
        correct_train += (torch.argmax(outputs, 1) == label).sum().item()

        total_loss += loss.item()

    # Learning Rate Scheduling
    before_lr = optimizer.param_groups[0]["lr"]
    scheduler.step()
    after_lr = optimizer.param_groups[0]["lr"]
    print("Epoch %d: SGD lr %.6f -> %.6f" % (epoch, before_lr, after_lr))


    # Calculate average training loss and accuracy
    average_loss = total_loss / len(train_loader)
    train_accuracy = correct_train / total_train

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}, Train Accuracy: {train_accuracy * 100:.2f}%')

# Save the trained model
torch.save(model.state_dict(), '/content/trained_model1.pth')

# Evaluate on the test set
model.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for batch in test_loader:
        label, features = batch['label'], batch['features']
        outputs = model(features)

        # Update test accuracy
        total_test += label.size(0)
        correct_test += (torch.argmax(outputs, 1) == label).sum().item()

test_accuracy = correct_test / total_test
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Print final training accuracy
final_train_accuracy = correct_train / total_train
print(f'Final Training Accuracy: {final_train_accuracy * 100:.2f}%')


Epoch 0: SGD lr 0.001000 -> 0.000900
Epoch [1/30], Loss: 35.1357, Train Accuracy: 89.29%
Epoch 1: SGD lr 0.000900 -> 0.000810
Epoch [2/30], Loss: 2.0763, Train Accuracy: 91.07%
Epoch 2: SGD lr 0.000810 -> 0.000729
Epoch [3/30], Loss: 0.5184, Train Accuracy: 83.93%
Epoch 3: SGD lr 0.000729 -> 0.000656
Epoch [4/30], Loss: 0.6037, Train Accuracy: 80.36%
Epoch 4: SGD lr 0.000656 -> 0.000590
Epoch [5/30], Loss: 0.6264, Train Accuracy: 75.89%
Epoch 5: SGD lr 0.000590 -> 0.000531
Epoch [6/30], Loss: 0.6348, Train Accuracy: 71.43%
Epoch 6: SGD lr 0.000531 -> 0.000478
Epoch [7/30], Loss: 0.6346, Train Accuracy: 71.43%
Epoch 7: SGD lr 0.000478 -> 0.000430
Epoch [8/30], Loss: 0.6388, Train Accuracy: 69.64%
Epoch 8: SGD lr 0.000430 -> 0.000387
Epoch [9/30], Loss: 0.6365, Train Accuracy: 66.96%
Epoch 9: SGD lr 0.000387 -> 0.000349
Epoch [10/30], Loss: 0.6339, Train Accuracy: 68.75%
Epoch 10: SGD lr 0.000349 -> 0.000314
Epoch [11/30], Loss: 0.6280, Train Accuracy: 67.86%
Epoch 11: SGD lr 0.000314 ->